In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.4 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Thu Nov 30 09:36:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer


load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

In [ ]:
model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
def response_gen(input_ids):
  temperature=0.7
  with torch.no_grad():
      generation_output = model_l.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=100,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer_l.decode(s)
  return output

In [ ]:
def ask_job_or_course(query, n):
  question = '''For the following question, return a probability in 2 decimal places whether it is asking about job or university course.
  Please return answer as a probability of the question asking about job.''' + query
  input = tokenizer_l(question, return_tensors='pt') # input = tokenizer_l(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")
  #print("embeds: ",input_ids,input_ids.shape)

  output =[]

  for i in range(n):
    response = response_gen(input_ids)
    output.append(response)
  return output

In [ ]:
q1 = "What does a software engineer do?"
q2 = "How to become a doctor?"
q3 = "What are the modules you can learn to become better at Accounting?"

iter = 1

a1 = ask_job_or_course(q1,iter)
a2 = ask_job_or_course(q2,iter)
a3 = ask_job_or_course(q3,iter)

print(q1, "\nPrediction: ", a1)
print("--------------------------------------------")
print(q2, "\nPrediction: ", a2)
print("--------------------------------------------")
print(q3, "\nPrediction: ", a3)


What does a software engineer do? 
Prediction:  ['</s>']
--------------------------------------------
How to become a doctor? 
Prediction:  ['\n\nQuestion: How to become a doctor?</s>']
--------------------------------------------
What are the modules you can learn to become better at Accounting? 
Prediction:  ['</s>']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
jobs = "drive/MyDrive/IR_Project/job_queries.txt"
jobs1= "/content/job_query.txt"

with open(jobs1) as f:
    lines = f.readlines()

FileNotFoundError: ignored

In [ ]:
lines

In [ ]:
# Open text file in write mode
text_file = open("job_query.txt", "w")

# Take content
content = questions

# Write content to file
n = text_file.write(content)

if n == len(content):
    print("Success! String written to text file.")
else:
    print("Failure! String not written to text file.")

# Close file
text_file.close()

Success! String written to text file.


In [ ]:
open_this = "/content/job_query.txt"

with open(open_this) as f:
    lines = f.readlines()

## VINCENT (Using the existing classification head API to train)

In [ ]:
from transformers import LlamaForSequenceClassification, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd

In [ ]:
with open('drive/MyDrive/IR_Project/job_query.txt', 'r') as file:
    lines = file.readlines()

# Create a DataFrame from the lines
job_df = pd.DataFrame({'query': lines})
job_df['label'] = 1.
# Display the DataFrame
job_df

,query,label
0,What are the primary responsibilities of a sof...,1.0
1,Can you outline the day-to-day tasks of a mark...,1.0
2,What are the key duties of a project manager?\n,1.0
3,Could you describe the roles and responsibilit...,1.0
4,What does a financial analyst typically do on ...,1.0
...,...,...
1995,Could you describe the various positions in gr...,1.0
1996,What types of jobs are available in social imp...,1.0
1997,Can you provide information on jobs in carbon ...,1.0
1998,What functions exist in environmental engineer...,1.0


In [ ]:
with open('drive/MyDrive/IR_Project/course_query.txt', 'r') as file:
    lines = file.readlines()

# Create a DataFrame from the lines
course_df = pd.DataFrame({'query': lines})
course_df['label'] = 0.
# Display the DataFrame
course_df

,query,label
0,What courses does NTU offer in the field of Co...,0.0
1,Can you provide information on the courses ava...,0.0
2,Tell me about the courses offered by SUTD in t...,0.0
3,Which courses are available at SMU for Economi...,0.0
4,Can you list the courses offered at SIT in the...,0.0
...,...,...
1995,Can you provide details on studying Medical Hu...,0.0
1996,What options are there for studying Community ...,0.0
1997,Explore possibilities for studying Arts and He...,0.0
1998,Tell me about programs in Healthcare Design at...,0.0


In [ ]:
input_df = pd.concat([job_df, course_df])

# Randomize (shuffle) the rows
input_df = input_df.sample(frac=1, random_state=42).reset_index(drop=True)
input_df

,query,label
0,What are the main tasks of a financial control...,1.0
1,What opportunities are there for studying Info...,0.0
2,What does a customer service manager usually o...,1.0
3,Tell me about programs in Anthropology at NUS.\n,0.0
4,Which courses are available at SMU for Informa...,0.0
...,...,...
3995,Could you describe the various jobs in the fit...,1.0
3996,Could you describe the various jobs in sustain...,1.0
3997,What does a healthcare policy researcher typic...,1.0
3998,Tell me about programs in Game Design and Deve...,0.0


In [ ]:
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

# modelulu = LlamaForSequenceClassification.from_pretrained(
#     model_name_l,
#     torch_dtype=torch.bfloat16,
#     load_in_8bit=load_in_8bit,
#     device_map="auto"
# )
# modelulu = AutoModelForSequenceClassification.from_pretrained(
#     "meta-llama/Llama-2-7b-hf",
#     torch_dtype=torch.bfloat16,
#     load_in_8bit=True,
#     device_map="auto"
# )
# llama-2-base

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, feature_extractor, max_tokens=50, train=True, verbose=False):
        self.df = df
        self.feature_extractor = feature_extractor
        self.train = train
        self.verbose = verbose
        self.max_tokens = max_tokens

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        if self.verbose: print("Index: ",index)
        row = self.df.iloc[index]

        label_features = row['query']
        label_features = tokenizer_l(label_features, return_tensors='pt')
        label_feature = label_features["input_ids"].to("cuda")
        label_feature = self.pad_or_truncate(label_feature, self.max_tokens)
        attention_mask = label_features["attention_mask"].to("cuda")
        label = torch.tensor(row['label']).view(1).to("cuda")

        if self.verbose: print("Features: ", label_feature.shape)
        if self.verbose: print("Label: ", label)
        if self.verbose: print("---------------------")
        return label_feature, attention_mask, label

    def pad_or_truncate(self, features, max_frames):
        # Pad or truncate the features to have max_frames
        num_frames = features.shape[1]
        if num_frames < max_frames:
            # Pad with zeros to reach max_frames
            padding = torch.zeros( features.shape[0], max_frames - num_frames).to("cuda")
            features = torch.cat((features, padding), dim=1)
        elif num_frames > max_frames:
            # Truncate to max_frames
            features = features[:, :max_frames]
        return features

In [ ]:
def process_df(df, train = True):
    if train == True:
      q_list = df['query'].unique()
      train_percentage = 0.8
      train_qs, val_qs = train_test_split(q_list, test_size=1 - train_percentage, random_state=42)

      train_df = df[df['query'].isin(train_qs)]
      val_df = df[df['query'].isin(val_qs)]

      return train_df, val_df
    else: return df

In [ ]:
class LinearModel(nn.Module):
    def __init__(self, dim_model, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden = nn.Linear(dim_model, dim_model)
        self.classification_head = nn.Linear(dim_model, 1)

    def forward(self, src: torch.tensor) -> torch.tensor:
        for i in range(self.num_layers):
          src = self.hidden(src)
          src = torch.sigmoid(src)

        src = self.classification_head(src)
        src = torch.sigmoid(src)
        src = src.squeeze(-1)

        return src

In [ ]:
from tqdm.auto import tqdm

In [ ]:
max_tokens = 50
num_layers = 2
modelulu = LinearModel(max_tokens, num_layers)
modelulu = modelulu.to("cuda")

optimizer = torch.optim.Adam(modelulu.parameters(), lr=1e-4)
criterion = torch.nn.BCELoss()
num_epochs = 50

train_df, val_df = process_df(input_df, True)

train_dataset = CustomDataset(train_df, tokenizer_l, max_tokens=max_tokens)
val_dataset = CustomDataset(val_df, tokenizer_l, max_tokens=max_tokens)

print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=None, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False)

verbose = False

# Training loop
for epoch in tqdm(range(num_epochs), position=0, leave=True):
    modelulu.train()
    running_loss = 0
    data_no = 0
    for inputs, attention_mask, labels in tqdm(train_loader, position=0, leave=True):
        optimizer.zero_grad()
        outputs = modelulu(inputs)
        loss = criterion(outputs, labels.float())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        data_no += 1
        if verbose: print(f"Batch no: {data_no}, Loss: {loss}")
    torch.save({
            'epoch': epoch+1,
            'model_state_dict': modelulu.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, 'heuristic_checkpoint.pt')
    print(f"Epoch: {epoch}, running loss: {running_loss}")
    # Evaluation loop
    modelulu.eval()
    print("Evaluation")
    for loaders in range(2):
      if loaders == 0:
        the_loader = train_loader
      else: the_loader = val_loader
      all_predicted_labels = []
      all_true_labels = []
      with torch.no_grad():
          for inputs, attention_mask, labels in the_loader:
              outputs = modelulu(inputs)

              predicted_class = (outputs >= 0.5).float()
              predicted_labels = predicted_class.cpu().numpy()

              all_predicted_labels.append(predicted_labels)
              all_true_labels.append(labels.cpu().numpy())
          f1 = f1_score(all_true_labels, all_predicted_labels)
          print(f"Validation F1 Score for epoch {epoch}: ", f1)

Train dataset length: 3207
Validation dataset length: 793


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/3207 [00:00<?, ?it/s]

Epoch: 0, running loss: 2200.2780208587646
Evaluation
Validation F1 Score for epoch 0:  0.5912698412698413
Validation F1 Score for epoch 0:  0.5855263157894737


  0%|          | 0/3207 [00:00<?, ?it/s]

...

Epoch: 47, running loss: 1304.324966987595
Evaluation
Validation F1 Score for epoch 47:  0.8229036295369212
Validation F1 Score for epoch 47:  0.8010139416983524


  0%|          | 0/3207 [00:00<?, ?it/s]

Epoch: 48, running loss: 1302.52477737749
Evaluation
Validation F1 Score for epoch 48:  0.8276729559748428
Validation F1 Score for epoch 48:  0.8020176544766708


  0%|          | 0/3207 [00:00<?, ?it/s]

Epoch: 49, running loss: 1299.3720155552728
Evaluation
Validation F1 Score for epoch 49:  0.824555936428794
Validation F1 Score for epoch 49:  0.8114143920595533


In [ ]:
def calculate_f1_score(true, pred):
    # Calculate true positives, false positives, and false negatives
    tp = sum(p == t == 1 for p, t in zip(pred, true))
    fp = sum(p == 1 and t == 0 for p, t in zip(pred, true))
    fn = sum(p == 0 and t == 1 for p, t in zip(pred, true))

    # Calculate precision, recall, and F1 score
    precision = tp / (tp + fp) if tp + fp != 0 else 0
    recall = tp / (tp + fn) if tp + fn != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return f1_score

In [ ]:
def process_tokenized(label_features, max_tokens=50):
    label_feature = label_features["input_ids"].to("cuda")
    num_frames = label_feature.shape[1]
    if num_frames < max_tokens:
        padding = torch.zeros(label_feature.shape[0], max_tokens - num_frames).to("cuda")
        label_feature = torch.cat((label_feature, padding), dim=1)
    elif num_frames > max_tokens:
        label_feature = label_feature[:, :max_tokens]
    return label_feature

In [ ]:
max_tokens = 50
num_layers = 2
modelulu = LinearModel(max_tokens, num_layers)
modelulu = modelulu.to("cuda")

checkpoint_path = 'heuristic_checkpoint.pt'

if os.path.exists(checkpoint_path):
    # Load checkpoint if it exists
    checkpoint = torch.load(checkpoint_path)
    # print(checkpoint)
    modelulu.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
query = "tell me more about healthcare industry"
tokenised_query = tokenizer_l(query, return_tensors='pt')
thefeat = process_tokenized(tokenised_query)
prob = modelulu(thefeat).item()
print(f"Probability of 'job': {prob}")

Probability of 'job': 0.7887129783630371


In [ ]:
train_df, val_df = process_df(input_df, True)

train_dataset = CustomDataset(train_df, tokenizer_l, max_tokens=max_tokens)
val_dataset = CustomDataset(val_df, tokenizer_l, max_tokens=max_tokens)

print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=None, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False)

print("Evaluation")
for loaders in range(2):
  if loaders == 0:
    the_loader = train_loader
  else: the_loader = val_loader
  all_predicted_labels = []
  all_true_labels = []
  with torch.no_grad():
      for inputs, attention_mask, labels in the_loader:
          outputs = modelulu(inputs)

          predicted_class = (outputs >= 0.5).float()
          predicted_labels = predicted_class.cpu().numpy()

          all_predicted_labels.append(predicted_labels)
          all_true_labels.append(labels.cpu().numpy())
      f1 = f1_score(all_true_labels, all_predicted_labels)
      print(f"Validation F1 Score: ", f1)

Train dataset length: 3207
Validation dataset length: 793
Evaluation
Validation F1 Score:  0.824555936428794
Validation F1 Score:  0.8114143920595533


In [ ]:
# fkin trash model dont run

optimizer = torch.optim.Adam(modelulu.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()#torch.nn.CrossEntropyLoss()
num_epochs = 4

train_df, val_df = process_df(input_df, True)

train_dataset = CustomDataset(train_df, tokenizer_l)
val_dataset = CustomDataset(val_df, tokenizer_l)

print(f"Train dataset length: {len(train_dataset)}")
print(f"Validation dataset length: {len(val_dataset)}")

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=None, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False)

verbose = False

# Training loop
for epoch in tqdm(range(num_epochs), position=0, leave=True):
    modelulu.train()
    running_loss = 0
    data_no = 0
    for inputs, attention_mask, labels in tqdm(train_loader, position=0, leave=True):
    #for batch_no, (inputs, attention_mask, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        if verbose: print(f"input shape: {inputs},{attention_mask}")
        if verbose: print(f"label: {labels}")
        outputs = modelulu(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        # loss, logits = outputs[:2]
        logits = outputs[0]
        if verbose: print(f"output logits: {logits.view(-1)}")
        # Calculate loss
        # loss = criterion(logits.view(-1, 2), labels)
        loss = criterion(logits.view(-1), labels.float().view(-1))

        if verbose: print(f"output loss: {loss}")
        print(f"Batch no: {data_no}, Loss: {loss}", end='\r')

        # loss = criterion(outputs, labels)

        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        data_no += 1
        if verbose: print(f"Batch no: {data_no}\nRunning loss: {running_loss}")
        if verbose: print(f"Batch no: {data_no}")
    torch.save({
            'epoch': epoch+1,
            'model_state_dict': modelulu.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, 'heuristic_checkpoint.pt')
    print(f"Epoch: {epoch}, running loss: {running_loss}")
    # Evaluation loop
    modelulu.eval()
    all_predicted_labels = []
    all_true_labels = []
    print("Evaluation")
    with torch.no_grad():
        for inputs, attention_mask, labels in val_loader:
            outputs = modelulu(input_ids=inputs, attention_mask=attention_mask)
            logits = outputs[0]

            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            predicted_class = torch.argmax(probabilities, dim=-1)
            predicted_labels = predicted_class.cpu().numpy()

            all_predicted_labels.extend(predicted_labels)
            all_true_labels.extend(labels.cpu().numpy())
        f1 = f1_score(all_true_labels, all_predicted_labels)
        print(f"F1 Score for epoch {epoch}: ", f1)

Train dataset length: 1602
Validation dataset length: 398


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1602 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


KeyboardInterrupt: ignored